In [38]:
from cmsis_svd.parser import SVDParser
import json
parser = SVDParser.for_packaged_svd("STMicro", "STM32F446x.svd")
dev = parser.get_device()
peripherals = dev.to_dict()['peripherals']


res = {}
for per in peripherals:
    res[per['name']] = {r['name']:dict(r, **{'address': per['base_address'] + r['address_offset']}) 
                        for r in per['registers']}

with open("hw_desc.json", "w") as f:
    json.dump(res, f, indent=4)